In [5]:
from crypto import cryptopuf
import numpy as np
from pypuf.pypuf import io
from pypuf.pypuf.metrics import uniqueness, bias
from pypuf.pypuf import attack
from pypuf.pypuf import metrics

In [6]:
# Uniqueness evaluation
seed = 31415
np.random.seed(seed)
puf_instances = []
for i in range(5):  # Create 5 PUF instances
    new_key = np.random.bytes(16).hex()  # New random key for each instance
    nonce = "000000000000000000000000"  # 96-bit nonce for TinyJAMBU
    crypto_puf = cryptopuf.CryptoPUF_TinyJAMBU(challenge_length=128, key=new_key, nonce=nonce)
    print(f"key is {new_key}")
    puf_instances.append(crypto_puf)

# Evaluate uniqueness using the `uniqueness()` function
uniqueness_score = uniqueness(puf_instances, seed=seed, N=1000)
print(f"Uniqueness score of the responses: {uniqueness_score}")

# Bias evaluation
bias_score = bias(puf_instances[0], seed=seed)
print(f"bias is {bias_score}")


Initializing CryptoPUF based on tinyjambu crypto core...
key is 3e5e54a5a71836fce336d8ac0d68ec14
Initializing CryptoPUF based on tinyjambu crypto core...
key is 650e80f8a8356891f28982e2866d7080
Initializing CryptoPUF based on tinyjambu crypto core...
key is 068ab8e8e35414031efdd6d2553f17bd
Initializing CryptoPUF based on tinyjambu crypto core...
key is 0e02997c66cc2af2706ff99e7056385d
Initializing CryptoPUF based on tinyjambu crypto core...
key is 883bbda4d30cdbbda69e7e663a5c1680
Uniqueness score of the responses: [0.9796]
bias is -0.004


In [7]:
N = 50000
print(f"Training with {N} CRPs...")

# Generate CRPs for the current iteration
crps = io.ChallengeResponseSet.from_simulation(crypto_puf, N=N, seed=2)

# Create and train the MLPAttack202 attack with the generated CRPs
attack_instance = attack.LRAttack2021(crps, seed=3, k=4, bs=1000, lr=.001, epochs=100)
attack_instance.fit()

# Evaluate the model's similarity score
model = attack_instance.model
score = metrics.similarity(crypto_puf, model, seed=4)
print(f"Predication accuracy of the trained LR model is with 50000 CRPs: {score}")


Training with 50000 CRPs...
Epoch 1/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4970 - loss: 0.6943 - val_accuracy: 0.4800 - val_loss: 0.6948
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5087 - loss: 0.6927 - val_accuracy: 0.4860 - val_loss: 0.6941
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5128 - loss: 0.6920 - val_accuracy: 0.4880 - val_loss: 0.6937
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 983us/step - accuracy: 0.5195 - loss: 0.6912 - val_accuracy: 0.4940 - val_loss: 0.6936
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 948us/step - accuracy: 0.5238 - loss: 0.6905 - val_accuracy: 0.4940 - val_loss: 0.6935
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - accuracy: 0.5235 - loss: 0.6897 - val_accuracy: 0.4960 - val_loss: 0.6936
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 920us/step - accuracy: 0.5253 - loss: 0.6889 - val_accuracy: 0.5200 - val_loss: 0.6937
Epoch 8/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step - accuracy: 0.5300 - l

In [8]:
# Create and train the MLPAttack202 attack with the generated CRPs
attack_instance = attack.MLPAttack2021(
crps, seed=3, net=[2 ** 4, 2 ** 5, 2 ** 4],
epochs=30, lr=.001, bs=1000, early_stop=.08
)
attack_instance.fit()

# Evaluate the model's similarity score
model = attack_instance.model
score = metrics.similarity(crypto_puf, model, seed=4)
print(f"Predication accuracy of the trained LR model is with 50000 CRPs: {score}")

Epoch 1/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4938 - loss: 0.7201 - val_accuracy: 0.4820 - val_loss: 0.6954
Epoch 2/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5133 - loss: 0.6927 - val_accuracy: 0.4800 - val_loss: 0.6952
Epoch 3/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5201 - loss: 0.6917 - val_accuracy: 0.4900 - val_loss: 0.6961
Epoch 4/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5256 - loss: 0.6908 - val_accuracy: 0.5000 - val_loss: 0.6976
Epoch 5/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5331 - loss: 0.6900 - val_accuracy: 0.4980 - val_loss: 0.6994
Epoch 6/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5362 - loss: 0.6892 - val_accuracy: 0.4720 - val_loss: 0.7015
Epoch 7/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5386 - loss: 0.6883 - val_accuracy: 0.4620 - val_loss: 0.7038
Epoch 8/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5447 - loss: 0.6874 - val_accuracy: 0.4760 - val_loss: